[Reunião de Condomínio](https://rachacuca.com.br/logica/problemas/reuniao-de-condominio/)

**Restrições:**

- O Comerciante está ao lado do morador que está cobrando mais Manutenção no condomínio.
- O morador de Verde está reclamando da Limpeza do prédio.
- O Joalheiro está exatamente à esquerda do condômino de 45 anos.
- César está exatamente à esquerda de quem está reclamando de Barulho.
- O Farmacêutico mora no apartamento 94.
- O homem de Verde está em algum lugar entre o homem de 35 anos e o homem de Vermelho, nessa ordem.
- O Arquiteto está ao lado do condômino que reclama do Barulho no prédio.
- Antônio está na quinta posição.
- O Comerciante está ao lado do homem mais velho.
- O morador de Branco está exatamente à direita do morador que reclama da Manutenção.
- O Joalheiro tem 40 anos.
- O condômino de camiseta Azul está ao lado do morador do apartamento 61.
- O Arquiteto está exatamente à direita de quem está reclamando da Garagem.
- Em uma das pontas está o morador do apartamento 123.
- Juan está exatamente à direita do Joalheiro.
- O morador de Azul está exatamente à esquerda de quem está reclamando da Manutenção.
- O condômino do apartamento 32 está exatamente à esquerda do Comerciante.
- O homem mais novo está em algum lugar à direita do morador de Azul.
- Vicente está em algum lugar entre o Rodolfo e o Juan, nessa ordem.
- O condômino mais velho está exatamente à esquerda do Farmacêutico.

In [1]:
using JuMP;
using GLPK;

Dentro da matriz `decisions`, a primeira dimensão se refere as características (cor da camiseta, nome, apartamento, reclamação, idade e profissão, nessa ordem), a segunda dimensão ao morador (1 a 5, nessa ordem) e a última se refere as opções de cada uma das características, na seguinte ordem:

| Cor da camiseta | Nome    | Apartamento | Reclamação | Idade   | Profissão    |
| --------------- | ------- | ----------- | ---------- | ------- | ------------ |
| Amarela         | Antônio | 15          | Barulho    | 30 anos | Arquiteto    |
| Azul            | César   | 32          | Garagem    | 35 anos | Comerciante  |
| Branca          | Juan    | 61          | Limpeza    | 40 anos | Farmacêutico |
| Verde           | Rodolfo | 94          | Manutenção | 45 anos | Joalheiro    |
| Vermelha        | Vicente | 123         | Segurança  | 50 anos | Produtor     |

In [2]:
function logic_test(restrictions::Vector{Bool} = [true for i in 1:20])
    model = Model(GLPK.Optimizer);
    @variable(model, decisions[1:6, 1:5, 1:5], base_name = "x", Bin);
    
    # para cada característica
    for i in 1:6
        # variando a pessoa ou o valor da característica
        for j in 1:5
            # a soma deve ser 1
            @constraint(model, sum(decisions[i, j, k] for k in 1:5) == 1);
            @constraint(model, sum(decisions[i, k, j] for k in 1:5) == 1);
        end
    end

    if restrictions[1]
        # O Comerciante está ao lado do morador que está cobrando mais Manutenção no condomínio.
        @constraint(model, decisions[6, 1, 2] <= decisions[4, 2, 4]);
        for i in 2:4
            @constraint(model, decisions[6, i, 2] <= decisions[4, i - 1, 4] + decisions[4, i + 1, 4]);
        end
        @constraint(model, decisions[6, 5, 2] <= decisions[4, 4, 4]);
    end
    
    if restrictions[2]
        # O morador de Verde está reclamando da Limpeza do prédio.
        for i in 1:5
            @constraint(model, decisions[1, i, 4] == decisions[4, i, 3]);
        end
    end
    
    if restrictions[3]
        # O Joalheiro está exatamente à esquerda do condômino de 45 anos.
        for i in 1:4
            @constraint(model, decisions[6, i, 4] == decisions[5, i + 1, 4]);
        end
        @constraint(model, decisions[6, 5, 4] == 0);
    end
    
    if restrictions[4]
        # César está exatamente à esquerda de quem está reclamando de Barulho.
        for i in 1:4
            @constraint(model, decisions[2, i, 2] == decisions[4, i + 1, 1]);
        end
        @constraint(model, decisions[2, 5, 2] == 0);
    end
    
    if restrictions[5]
        # O Farmacêutico mora no apartamento 94.
        for i in 1:5
            @constraint(model, decisions[6, i, 3] == decisions[3, i, 4]);
        end
    end
    
    if restrictions[6]
        # O homem de Verde está em algum lugar entre o homem de 35 anos e o homem de Vermelho, nessa ordem.
        @constraint(model, decisions[1, 1, 4] == 0); # O de verde não pode ser
        @constraint(model, decisions[1, 5, 4] == 0); # o primeiro nem o último
        @constraint(model, decisions[5, 4, 2] == 0); # O de 35 anos não pode ser
        @constraint(model, decisions[5, 5, 2] == 0); # o quarto nem o quinto
        @constraint(model, decisions[1, 1, 5] == 0); # O de vermelho não pode ser o
        @constraint(model, decisions[1, 2, 5] == 0); # primeiro nem o segundo
        @constraint(model, decisions[5, 1, 2] + decisions[1, 5, 5] >= sum(decisions[1, i, 4] for i in 2:4) - 1);
        @constraint(model, decisions[5, 1, 2] + decisions[1, 4, 5] >= decisions[1, 2, 4] + decisions[1, 3, 4] - 1);
        @constraint(model, decisions[5, 1, 2] + decisions[1, 3, 5] >= decisions[1, 2, 4]);
        @constraint(model, decisions[5, 2, 2] + decisions[1, 5, 5] >= decisions[1, 3, 4] + decisions[1, 4, 4] - 1);
        @constraint(model, decisions[5, 2, 2] + decisions[1, 4, 5] >= decisions[1, 3, 4] - 1);
        @constraint(model, decisions[5, 3, 2] + decisions[1, 5, 5] >= decisions[1, 4, 4]);
        for i in 1:5
            @constraint(model, decisions[1, i, 4] + decisions[5, i, 2] + decisions[1, i, 5] <= 1);
        end
    end
    
    if restrictions[7]
        # O Arquiteto está ao lado do condômino que reclama do Barulho no prédio.
        @constraint(model, decisions[6, 1, 1] <= decisions[4, 2, 1]);
        for i in 2:4
            @constraint(model, decisions[6, i, 1] <= decisions[4, i - 1, 1] + decisions[4, i + 1, 1]);
        end
        @constraint(model, decisions[6, 5, 1] <= decisions[4, 4, 1]);
    end
    
    if restrictions[8]
        # Antônio está na quinta posição.
        @constraint(model, decisions[2, 5, 1] == 1);
    end
    
    if restrictions[9]
        # O Comerciante está ao lado do homem mais velho.
        @constraint(model, decisions[6, 1, 2] <= decisions[5, 2, 5]);
        for i in 2:4
            @constraint(model, decisions[6, i, 2] <= decisions[5, i - 1, 5] + decisions[5, i + 1, 5]);
        end
        @constraint(model, decisions[6, 5, 2] <= decisions[5, 4, 5]);
    end
    
    if restrictions[10]
        # O morador de Branco está exatamente à direita do morador que reclama da Manutenção.
        for i in 1:4
            @constraint(model, decisions[1, i + 1, 3] == decisions[4, i, 4]);
        end
        @constraint(model, decisions[1, 1, 3] == 0);
    end
    
    if restrictions[11]
        # O Joalheiro tem 40 anos.
        for i in 1:5
            @constraint(model, decisions[6, i, 4] == decisions[5, i, 3]);
        end
    end
    
    if restrictions[12]
        # O condômino de camiseta Azul está ao lado do morador do apartamento 61.
        @constraint(model, decisions[1, 1, 2] <= decisions[3, 2, 3]);
        for i in 2:4
            @constraint(model, decisions[1, i, 2] <= decisions[3, i - 1, 3] + decisions[3, i + 1, 3]);
        end
        @constraint(model, decisions[1, 5, 2] <= decisions[3, 4, 3]);
    end
    
    if restrictions[13]
        # O Arquiteto está exatamente à direita de quem está reclamando da Garagem.
        for i in 1:4
            @constraint(model, decisions[6, i + 1, 1] == decisions[4, i, 2]);
        end
        @constraint(model, decisions[6, 1, 1] == 0);
    end
    
    if restrictions[14]
        # Em uma das pontas está o morador do apartamento 123.
        @constraint(model, decisions[3, 1, 5] + decisions[3, 5, 5] == 1);
    end
    
    if restrictions[15]
        # Juan está exatamente à direita do Joalheiro.
        for i in 1:4
            @constraint(model, decisions[2, i + 1, 3] == decisions[6, i, 4]);
        end
        @constraint(model, decisions[2, 1, 3] == 0);
    end
    
    if restrictions[16]
        # O morador de Azul está exatamente à esquerda de quem está reclamando da Manutenção.
        for i in 1:4
            @constraint(model, decisions[1, i, 2] == decisions[4, i + 1, 4]);
        end
        @constraint(model, decisions[1, 5, 2] == 0);
    end
    
    if restrictions[17]
        # O condômino do apartamento 32 está exatamente à esquerda do Comerciante.
        for i in 1:4
            @constraint(model, decisions[3, i, 2] == decisions[6, i + 1, 2]);
        end
        @constraint(model, decisions[3, 5, 2] == 0);
    end
    
    if restrictions[18]
        # O homem mais novo está em algum lugar à direita do morador de Azul.
        @constraint(model, decisions[1, 5, 2] == 0); # o cara de azul não pode ser o último
        @constraint(model, decisions[5, 1, 1] == 0); # e o mais novo não pode ser o primeiro
        @constraint(model, decisions[1, 1, 2] >= sum(decisions[5, i, 1] for i in 2:5) - 1);
        @constraint(model, decisions[1, 2, 2] >= sum(decisions[5, i, 1] for i in 3:5) - 1);
        @constraint(model, decisions[1, 3, 2] >= sum(decisions[5, i, 1] for i in 4:5) - 1);
        @constraint(model, decisions[1, 4, 2] >= decisions[5, 5, 1] - 1);
        @constraint(model, decisions[1, 3, 2] + decisions[5, 2, 1] <= 1);
        @constraint(model, decisions[1, 4, 2] + decisions[5, 2, 1] <= 1);
        @constraint(model, decisions[1, 4, 2] + decisions[5, 3, 1] <= 1);

        for i in 1:5
            @constraint(model, decisions[1, i, 2] + decisions[5, i, 1] <= 1);
        end
    end
    
    if restrictions[19]
        # Vicente está em algum lugar entre o Rodolfo e o Juan, nessa ordem.
        @constraint(model, decisions[2, 1, 5] == 0); # Vicente não pode ser
        @constraint(model, decisions[2, 5, 5] == 0); # o primeiro nem o último
        @constraint(model, decisions[2, 4, 4] == 0); # Rodolfo não pode ser
        @constraint(model, decisions[2, 5, 4] == 0); # o quarto nem o quinto
        @constraint(model, decisions[2, 1, 3] == 0); # Juan não pode ser o
        @constraint(model, decisions[2, 2, 3] == 0); # primeiro nem o segundo
        @constraint(model, decisions[2, 1, 4] + decisions[2, 5, 3] >= sum(decisions[2, i, 5] for i in 2:4) - 1);
        @constraint(model, decisions[2, 1, 4] + decisions[2, 4, 3] >= decisions[2, 2, 5] + decisions[2, 3, 5] - 1);
        @constraint(model, decisions[2, 1, 4] + decisions[2, 3, 3] >= decisions[2, 2, 5]);
        @constraint(model, decisions[2, 2, 4] + decisions[2, 5, 3] >= decisions[2, 3, 5] + decisions[2, 4, 5] - 1);
        @constraint(model, decisions[2, 2, 4] + decisions[2, 4, 3] >= decisions[2, 3, 5] - 1);
        @constraint(model, decisions[2, 3, 4] + decisions[2, 5, 3] >= decisions[2, 4, 5]);
    end
    
    if restrictions[20]
        # O condômino mais velho está exatamente à esquerda do Farmacêutico.
        for i in 1:4
            @constraint(model, decisions[5, i, 5] == decisions[6, i + 1, 3]);
        end
        @constraint(model, decisions[5, 5, 5] == 0);
    end

    # @objective(model, Max, sum(decisions));
    optimize!(model);
    return JuMP.value.(decisions);
end;

In [3]:
all_restrictions = [["O Comerciante está ao lado do morador que está cobrando mais Manutenção no condomínio.",
                     [["Comerciante", "Manutenção", [-1, 1]]]],
                    ["O morador de Verde está reclamando da Limpeza do prédio.",
                     [["Verde", "Limpeza", 0:0]]],
                    ["O Joalheiro está exatamente à esquerda do condômino de 45 anos.",
                     [["Joalheiro", "45 anos", 1:1]]],
                    ["César está exatamente à esquerda de quem está reclamando de Barulho.",
                     [["César", "Barulho", 1:1]]],
                    ["O Farmacêutico mora no apartamento 94.",
                     [["Farmacêutico", "94", 0:0]]],
                    ["O homem de Verde está em algum lugar entre o homem de 35 anos e o homem de Vermelho, nessa ordem.",
                     [["35 anos", "Verde", 1:3], ["Verde", "Vermelha", 1:3]]],
                    ["O Arquiteto está ao lado do condômino que reclama do Barulho no prédio.",
                     [["Arquiteto", "Barulho", [-1, 1]]]],
                    ["Antônio está na quinta posição.",
                     [["Antônio", "Morador 5", 0:0]]],
                    ["O Comerciante está ao lado do homem mais velho.",
                     [["Comerciante", "50 anos", [-1, 1]]]],
                    ["O morador de Branco está exatamente à direita do morador que reclama da Manutenção.",
                     [["Branca", "Manutenção", (-1):(-1)]]],
                    ["O Joalheiro tem 40 anos.",
                     [["Joalheiro", "40 anos", 0:0]]],
                    ["O condômino de camiseta Azul está ao lado do morador do apartamento 61.",
                     [["Azul", "61", [-1, 1]]]],
                    ["O Arquiteto está exatamente à direita de quem está reclamando da Garagem.",
                     [["Arquiteto", "Garagem", (-1):(-1)]]],
                    ["Em uma das pontas está o morador do apartamento 123.",
                     [["123", "Morador 3", [-2, 2]]]],
                    ["Juan está exatamente à direita do Joalheiro.",
                     [["Juan", "Joalheiro", (-1):(-1)]]],
                    ["O morador de Azul está exatamente à esquerda de quem está reclamando da Manutenção.",
                     [["Azul", "Manutenção", 1:1]]],
                    ["O condômino do apartamento 32 está exatamente à esquerda do Comerciante.",
                     [["32", "Comerciante", 1:1]]],
                    ["O homem mais novo está em algum lugar à direita do morador de Azul.",
                     [["30 anos", "Azul", (-4):(-1)]]],
                    ["Vicente está em algum lugar entre o Rodolfo e o Juan, nessa ordem.",
                     [["Rodolfo", "Vicente", 1:3], ["Vicente", "Juan", 1:3]]],
                    ["O condômino mais velho está exatamente à esquerda do Farmacêutico.",
                     [["50 anos", "Farmacêutico", 1:1]]]];

In [4]:
function print_solution(solution)
    lines = [];
    header = "| Característica  |";
    sep    = "| --------------- |";
    for i in 1:5
        header *= " Morador " * string(i) * "       |" ;
        sep *= " --------------- |";
    end
    push!(lines, header);
    push!(lines, sep);
    
    caracteristicas = ["| Cor da camiseta |", "| Nome            |", "| Apartamento     |",
                       "| Reclamação      |", "| Idade           |", "| Profissão       |"];
    
    cores           = [ " Amarela         |",  " Azul            |",  " Branca          |",
                        " Verde           |",  " Vermelha        |"];
    
    nomes           = [ " Antônio         |",  " César           |",  " Juan            |",
                        " Rodolfo         |",  " Vicente         |"];
    
    apartamentos    = [ " 15              |",  " 32              |",  " 61              |",
                        " 94              |",  " 123             |"];
    
    reclamacoes     = [ " Barulho         |",  " Garagem         |",  " Limpeza         |",
                        " Manutenção      |",  " Segurança       |"];
    
    idades          = [ " 30 anos         |",  " 35 anos         |",  " 40 anos         |",
                        " 45 anos         |",  " 50 anos         |"];
    
    profissoes      = [ " Arquiteto       |",  " Comerciante     |",  " Farmacêutico    |",
                        " Joalheiro       |",  " Produtor        |"];
    
    println(header);
    println(sep);
    dados = [cores, nomes, apartamentos, reclamacoes, idades, profissoes];
    for i in 1:6
        line = caracteristicas[i];
        for j in 1:5
            for k in 1:5
                if solution[i, j, k] == 1
                    line *= dados[i][k];
                end
            end
        end
        println(line);
        push!(lines, line);
    end
    return lines;
end;

function find_position(solution, key)
    for line in solution
        if occursin(key, line)
            positions = findall("|", line);
            for i in 2:length(positions)
                if ~occursin(key, line[positions[i][1]:length(line)])
                    return i - 2;
                end
            end
        end
    end
end;

function relative_position(solution, key1, key2)
    a = find_position(solution, key1)[1];
    b = find_position(solution, key2)[1];
    return (b - a);
end;

function verify_solution(solution, restrictions)
    solution = print_solution(solution);
    println();
    for restriction in restrictions
        ok = true;
        for condition in restriction[2]
            if ~(relative_position(solution, condition[1], condition[2]) in condition[3])
                ok = false;
            end
        end
        if ok
            println("✅ " * restriction[1]);
        else
            println("❌ " * restriction[1]);
        end
    end
end;

In [5]:
verify_solution(logic_test(), all_restrictions);

| Característica  | Morador 1       | Morador 2       | Morador 3       | Morador 4       | Morador 5       |
| --------------- | --------------- | --------------- | --------------- | --------------- | --------------- |
| Cor da camiseta | Amarela         | Verde           | Azul            | Vermelha        | Branca          |
| Nome            | Rodolfo         | Vicente         | Juan            | César           | Antônio         |
| Apartamento     | 123             | 32              | 15              | 61              | 94              |
| Reclamação      | Segurança       | Limpeza         | Garagem         | Manutenção      | Barulho         |
| Idade           | 35 anos         | 40 anos         | 45 anos         | 50 anos         | 30 anos         |
| Profissão       | Produtor        | Joalheiro       | Comerciante     | Arquiteto       | Farmacêutico    |

✅ O Comerciante está ao lado do morador que está cobrando mais Manutenção no condomínio.
✅ O morador de Verde está recl

In [6]:
restrictions = rand(Bool, length(all_restrictions));
verify_solution(logic_test(restrictions), all_restrictions[restrictions]);

| Característica  | Morador 1       | Morador 2       | Morador 3       | Morador 4       | Morador 5       |
| --------------- | --------------- | --------------- | --------------- | --------------- | --------------- |
| Cor da camiseta | Amarela         | Azul            | Branca          | Verde           | Vermelha        |
| Nome            | Antônio         | César           | Rodolfo         | Vicente         | Juan            |
| Apartamento     | 15              | 32              | 61              | 123             | 94              |
| Reclamação      | Garagem         | Manutenção      | Barulho         | Limpeza         | Segurança       |
| Idade           | 30 anos         | 35 anos         | 45 anos         | 40 anos         | 50 anos         |
| Profissão       | Produtor        | Arquiteto       | Comerciante     | Joalheiro       | Farmacêutico    |

✅ O Comerciante está ao lado do morador que está cobrando mais Manutenção no condomínio.
✅ César está exatamente à esqu